In [1]:
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset

# Network

In [2]:
class traffic_convnet(nn.Module):
    
    def __init__(self, channels):
        super(traffic_convnet, self).__init__()
        self.channels = channels
        self.num_links = channels
        
        # Input: [BatchSize, Channels, H, W]
        
        # define backbone:
        self.bn1 = nn.BatchNorm2d(self.channels)
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=self.channels,
                out_channels=30,
                kernel_size=3,
                stride=1,
                padding_mode='zeros',
                padding=1
            ),  # ! no padding
            nn.ReLU()
        )
        self.avgpool1 = nn.AvgPool2d(
            kernel_size=2,
            stride=1
        )
        
        self.bn2 = nn.BatchNorm2d(30)
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=30,
                out_channels=30,
                kernel_size=3,
                stride=1,
                padding_mode='zeros',
                padding=1
            ),  # ! no padding
            nn.ReLU()
        )
        self.avgpool2 = nn.AvgPool2d(
            kernel_size=2,
            stride=1
        )
        
        self.bn3 = nn.BatchNorm2d(30)
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Sequential(
            nn.Linear(5760, self.num_links*6),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.conv1(x)
        pad1 = (1, 0, 1, 0)  ###
        x = F.pad(x, pad1)
        x = self.avgpool1(x)
        x = self.bn2(x)
        x = self.conv2(x)
        x = F.pad(x, pad1)
        x = self.avgpool2(x)
        x = self.bn3(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc1(x)
        #x = x.view(-1, self.num_links, x.shape[1]//self.num_links)
        return x

In [3]:
def cal_L2_dist(total):
    try:
        total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0-total1)**2).sum(2)
    except:
        total_cpu = total.to('cpu')
        len_ = total_cpu.shape[0]
        L2_distance = torch.zeros([len_, len_], device=total_cpu.device.type)
        for i in range(total_cpu.shape[1]):
            total0 = total_cpu[:, i].unsqueeze(0).expand(int(total_cpu.size(0)), int(total_cpu.size(0)))
            total1 = total_cpu[:, i].unsqueeze(1).expand(int(total_cpu.size(0)), int(total_cpu.size(0)))
            L2_dist = (total0 - total1)**2
            L2_distance += L2_dist
            
    return L2_distance


def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    #source = source.cpu()
    #target = target.cpu()
    n_samples = int(source.size()[0])+int(target.size()[0])  # number of samples
    total = torch.cat([source, target], dim=0)
    L2_distance = cal_L2_dist(total)
                       
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    return sum(kernel_val).to(total.device.type)  #/len(kernel_val)


def mmd_rbf_accelerate(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
        kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    loss = 0
    for i in range(batch_size):
        s1, s2 = i, (i+1) % batch_size
        t1, t2 = s1 + batch_size, s2 + batch_size
        loss += kernels[s1, s2] + kernels[t1, t2]
        loss -= kernels[s1, t2] + kernels[s2, t1]
    return loss / float(batch_size)

def mmd_rbf_noaccelerate(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
                              kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    XX = kernels[:batch_size, :batch_size]
    YY = kernels[batch_size:, batch_size:]
    XY = kernels[:batch_size, batch_size:]
    YX = kernels[batch_size:, :batch_size]
    loss = torch.mean(XX + YY - XY -YX)
    return loss

In [4]:
class traffic_dannet(nn.Module):
    
    def __init__(self, channels, if_mmd=False):
        super(traffic_dannet, self).__init__()
        self.sharedNet = traff_convnet(channels)
        self.num_links = channels
        self.if_mmd = if_mmd
        
    def forward(self, source, target):
        mmd_loss = 0
        source_out = self.sharedNet(source)
        
        if self.if_mmd == True:
            target_out = self.sharedNet(target)  # shared net is resnet 50
            #loss += mmd.mmd_rbf_accelerate(source, target)
            mmd_loss += mmd_rbf_noaccelerate(source_out, target_out)
            
            source_out = source_out.view(
                -1, self.num_links, source_out.shape[1]//self.num_links
            )
            target_out = target_out.view(
                -1, self.num_links, target_out.shape[1]//self.num_links
            )

            return source_out, target_out, mmd_loss
        
        else:
            source_out = source_out.view(
                -1, self.num_links, source_out.shape[1]//self.num_links
            )
            return source_out, source_out, 0

def traff_convnet(channels):
    model = traffic_convnet(channels)
    return model

In [5]:
def mape_loss_func(preds, labels):
    try:
        if preds.device.type == 'cuda':
            preds = preds.cpu().detach().numpy()
        if labels.device.type == 'cuda':
            labels = labels.cpu().detach().numpy()
    except:
        None
        
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    try:
        if preds.device.type == 'cuda':
            preds = preds.cpu().detach().numpy()
        if labels.device.type == 'cuda':
            labels = labels.cpu().detach().numpy()
    except:
        None
        
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    try:
        if preds.device.type == 'cuda':
            preds = preds.cpu().detach().numpy()
        if labels.device.type == 'cuda':
            labels = labels.cpu().detach().numpy()
    except:
        None
        
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

# Load Data

In [6]:
def sliding_window(near_road, flow, k, t_p, t_input, t_pre, batch_size, train_prop):
    
    time_gran = flow.shape[1]//t_p
    num_links = near_road.shape[0]
    
    image = []
    for i in range(np.shape(near_road)[0]):
        road_id = []
        for j in range(k):
            # near_road = np.argsort(distance)
            # distance[near_road[n]] = nth smallest of "distance"
            road_id.append(near_road[i][j])
        image.append(flow[road_id, :])
    image = np.array(image)  # shape = [num_links, k, t_total]
    image = np.transpose(image, (1, 2, 0))  # [num_links, k, t_total] --> [k, t_total, num_links]

    image3 = []
    label = []

    # sliding window
    for i in range(1, t_p):  # ? interval: [1, 61]
        for j in range(time_gran-t_input-t_pre):
            image3.append(image[:, i*time_gran+j        :i*time_gran+j+t_input      , :])
            label.append(flow  [:, i*time_gran+j+t_input:i*time_gran+j+t_input+t_pre])
    
    image3 = np.array(image3)
    image3 = np.transpose(image3, (0, 3, 1, 2))  # adjusted for pytorch con net
    label = np.asarray(label)
    

    # 划分前80%数据为训练集，最后20%数据为测试集
    image_train = image3[:int(np.shape(image3)[0]*train_prop)]
    image_val = image3[int(np.shape(image3)[0]*train_prop):]
    label_train = label[:int(np.shape(label)[0]*train_prop)]
    label_val = label[int(np.shape(label)[0]*train_prop):]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_train = torch.tensor(image_train, dtype=torch.float32).to(device)
    label_train = torch.tensor(label_train, dtype=torch.float32).to(device)
    image_val = torch.tensor(image_val, dtype=torch.float32).to(device)
    label_val = torch.tensor(label_val, dtype=torch.float32).to(device)
    
    ###
    if image_train.shape[0] != 2956:
        temp = torch.cat((image_train, image_train), axis=0)
        image_train = torch.cat((temp, temp), axis=0)
        temp = torch.cat((label_train, label_train), axis=0)
        label_train = torch.cat((temp, temp), axis=0)
    ###
    
    # data loader
    #train_dataset = TensorDataset(image_train, label_train)
    #train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
    
    return num_links, image_train, label_train, image_val, label_val

In [7]:
src_near_road = np.array(pd.read_csv(
    '../data/network/2small_network_nearest_road_id.csv', header=0
))

tar_near_road = np.array(pd.read_csv('../data/network/small_network_nearest_road_id.csv', header=0))

src_flow = np.array(
    pd.read_csv('../data/network/2small_network_flow.csv', header=0).iloc[:, 2:]
)
tar_flow = np.array(pd.read_csv('../data/network/small_network_flow.csv', header=0).iloc[:, 2:])

In [8]:
k = 4 # K个路段
t_p = 25  # number of days
t_input = 48  # 参数t_input为输入时间窗(5min颗粒度)
t_pre = 6  # 参数t_pre为预测时间窗(5min颗粒度)
batch_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式
src_num_links, src_train_data, src_train_label, src_image_val, src_label_val = sliding_window(
    src_near_road, src_flow, k, t_p, t_input, t_pre, batch_size, train_prop=1
)

tar_num_links, tar_train_data, tar_train_label, tar_val_data, tar_val_label = sliding_window(
    tar_near_road, tar_flow, k, t_p, t_input, t_pre, batch_size, train_prop=0.1
)

src_train_dataset = TensorDataset(src_train_data, src_train_label)
tar_train_dataset = TensorDataset(tar_train_data, tar_train_label)

src_train_loader = torch.utils.data.DataLoader(src_train_dataset, batch_size, shuffle=True)
tar_train_loader = torch.utils.data.DataLoader(tar_train_dataset, batch_size, shuffle=True)

#num_links, train_loader, image_val, label_val = sliding_window(near_road, flow, k, t_p, t_input, t_pre, batch_size)

In [9]:
traff_dannet = torch.load('../model/net_convnet_final.pth')

traff_dannet.eval()

val_out, val_out, mmd_loss = traff_dannet(tar_val_data, tar_val_data)
#val_out_denormed = denorm_data(val_out, tar_min, tar_max)
#tar_val_label_denormed = denorm_data(tar_val_label, tar_min, tar_max)

print('MAPE: %.5f'%mape_loss_func(val_out, tar_val_label))
print('SMAPE: %.5f'%smape_loss_func(val_out, tar_val_label))
print('MAE: %.5f'%mae_loss_func(val_out, tar_val_label))

MAPE: 0.88790
SMAPE: 0.54527
MAE: 14.09851
